# HW4 - Group 10

## 1) Hashing Task

In [1]:
import pandas as pd
import numpy as np
import math
import time
import sympy

we make a list of sorted passwords which we are gonna use later for finding the number of false positives 

In [2]:
sortpass = sorted(open("passwords1.txt").read().splitlines())

In [3]:
#with open("C:\\Users\\Nino\\ADM HW4\\passwords1.txt", 'r') as file :
    #df = pd.DataFrame(file.read().split("\n"), columns = ["pass1"])

In [4]:
#with open("C:\\Users\\Nino\\ADM HW4\\passwords2.txt", 'r') as file :
    #df["pass2"] = file.read().split("\n")

In [5]:
#len(df["pass1"])

In [6]:
#len(df["pass2"])

for implementing bloom filter, first we need to difine indipendent hash functions. Each hash function should get a string(passwords here) as number and return an integer. We decided to define 4 hash functions and build the bloom filter based on them. In the first hash function we get the ASCII code of password by multiplying the ASCII code of each character with 37 power the position of the character. Then to get indexes from these hash numbers we calculate the modular value over the size of bit array. We decided to choose the size of bit array 6 times bigger than number of elements in the password1 file, and in order to have uniformity in our bloom filter we pick the first prime number after 6*n which is 600000001 (explanation about choosing this number is in the later sections)

In [7]:
def hash_fun1 (string):
    h = [ord(i) for i in string]
    hashnumber = sum([(37**h.index(i))*i for i in h])% 600000001
    return(hashnumber)
    
    

In [8]:
hash_fun1('b*ZB/egd=!XcvV>K(qY@')

544418264

In the first hash function we get the ASCII code of each password by multiplying the ASCII code of each character with 37 power the difference of  character's position and 20. To get indexes from this number, we calculate the modular value over the size of bit array

In [9]:
def hash_fun2 (string):
    h = [ord(i) for i in string]
    hashnumber = sum([(37**(20 - h.index(i)))*i for i in h])% 600000001
    return(hashnumber)

In [10]:
hash_fun2('b*ZB/egd=!XcvV>K(qY@')

265687178

The third hash function concatenate ASCII codes of each 4 characters of the password, then calculate the sum of that value and at last return the modualar value over the size of bit array

In [11]:
def hash_fun3 (string) :
    h = [ord(i) for i in string]
    hashnumber = 0
    for i in range(0,5):
        s="".join(map(str, h[4*i : 4*i + 4]))
        try :
            hashnumber += int(s)
        except :
            pass
        hashnumber = hashnumber % 600000001
        
    return(hashnumber)    

In [12]:
hash_fun3('b*ZB/egd=!XcvV>K(qY@')

380876225

The fourth hash function concatenate ASCII codes of each 5 characters of the password, then calculate the sum of that value and at last return the modualar value over the size of bit array

In [13]:
def hash_fun4 (string) :
    h = [ord(i) for i in string]
    hashnumber = 0
    for i in range(0,4):
        s="".join(map(str, h[5*i : 5*i + 5]))
        try :
            hashnumber += int(s)
        except :
            pass
        hashnumber = hashnumber % 600000001
        
    return(hashnumber)  

In [14]:
hash_fun4('b*ZB/egd=!XcvV>K(qY@')

266238431

In [15]:
n = 100000000
m = (sympy.nextprime(6*n))
#print(m)
bfilter = np.zeros(m,int)

Now, it's time to implement our bloom filter. we take an array of size m = 600000001 with all elements equal to 0, and for each password in password1 file we update the array

In [16]:
# this function change 0 to 1 in our bit array in indexes of 4 hashnumbers we get for each password
def update_filter(array) :
    with open("passwords1.txt", 'r') as file :
            for line in file :
                array[hash_fun1(line)] = 1
                array[hash_fun2(line)] = 1
                array[hash_fun3(line)] = 1
                array[hash_fun4(line)] = 1
   

We create a Binary search function which we are gonna use for finding the number of False positives for the bonus part. 

In [17]:
# This function will return 1 if the value exist in the array and 0 if the value does not exist
def BinarySearch(array, val):
    first = 0
    last = len(array)-1
    while (first <= last) :
        mid = (first+last)//2
        if array[mid] == val:
            return(1)
        
        elif val < array[mid]:
            last = mid -1
            
        else:
            first = mid +1
    return (0)

Now, we want to implement a search filter, which for each password check whether all 4 hash numbers exist in the bit array and if it does it would increase the counter by 1. Also we decided to do the bonus part here, so for each password in password2 file we do binary search in the sorted list of password1 to see if that password exist or not. We used binary search because it's time efficient

In [18]:
def search_filter(array) :
    counter = 0
    Truepos = 0
    with open("passwords2.txt", 'r') as file :
        for line in file :
            if(array[hash_fun1(line)] == 1  and  array[hash_fun2(line)] == 1
              and array[hash_fun3(line)] == 1 and array[hash_fun4(line)] == 1) :
                counter += 1
                Truepos += BinarySearch(sortpass, line.replace("\n","")) 
                
    return(counter,Truepos)       

For deciding about the size of our bit array, I used the formula k = m * ln(2)/n for k=4 and n = 100000000. This equation gives approximate 6*n for size of our bit array. For having more uniformity in our filter, I pick the next prime numer aftr 6 * n as the size of our bit array




In [19]:
def bloom_filter(array):
    k = 4 #number of hash functions
    n = 100000000 #elemnts in bloom filter
    m = 600000001 #size of bit array
    p = pow(1 - math.exp(-k / (m / n)), k) #probability of false positive
    start = time.time()
    update_filter(array)
    duplicates = search_filter(array)[0]
    Truepositive = search_filter(array)[1]
    end = time.time()
    runtime = end - start
    print('Number of hash function used: ', k)
    print('Number of duplicates detected: ', duplicates)
    print('Probability of false positives: ', p)
    print('Number of True duplicates is : ', Truepositive)
    print('Number of false positives is : ' , duplicates - Truepositive )
    print('Execution time: ', runtime)
    
    

In [20]:
bloom_filter(bfilter)

Number of hash function used:  4
Number of duplicates detected:  15402920
Probability of false positives:  0.05605667057746283
Number of True duplicates is :  14000000
Number of false positives is :  1402920
Execution time:  6996.77089548111
